In [88]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [89]:
bulgaria_gdf = gpd.read_file("data/bg.json")

In [90]:
bulgaria_gdf["name"] = bulgaria_gdf["name"].str.lower()
bulgaria_gdf = bulgaria_gdf.replace("grad sofiya", "sofia-grad")

## Get Open Fertility data

In [91]:
raw_population_df = pd.read_csv("data/report_1746469196230.csv", sep=";")

In [92]:
formatted_raw_population_df = raw_population_df.rename(
    columns={"Unnamed: 0": "YEAR", "Unnamed: 1": "name", "Number": "POPULATION"}
).iloc[2:]
formatted_raw_population_df["YEAR"] = formatted_raw_population_df["YEAR"].astype(int)
formatted_raw_population_df["POPULATION"] = formatted_raw_population_df[
    "POPULATION"
].astype(int)
formatted_raw_population_df["name"] = formatted_raw_population_df["name"].str.lower()

In [93]:
formatted_raw_population_df = formatted_raw_population_df[
    ["YEAR", "name", "POPULATION"]
]
population_2015_df = formatted_raw_population_df[
    formatted_raw_population_df["YEAR"] == 2015
]
population_2024_df = formatted_raw_population_df[
    formatted_raw_population_df["YEAR"] == 2024
]

In [94]:
population_df = population_2015_df.merge(
    population_2024_df, on="name", how="inner", suffixes=("_2015", "_2024")
)
population_df["DIFF"] = (
    population_df["POPULATION_2024"] - population_df["POPULATION_2015"]
)
population_df["PERCENT_DIFF"] = (
    (
        (population_df["POPULATION_2024"] - population_df["POPULATION_2015"])
        / population_df["POPULATION_2015"]
    )
    * 100
).round(decimals=1)

In [95]:
population_df = population_df[
    ["name", "POPULATION_2015", "POPULATION_2024", "DIFF", "PERCENT_DIFF"]
]

## Merge data and export

In [96]:
population_gdf = bulgaria_gdf.merge(population_df, on="name", how="right")

In [97]:
population_gdf = population_gdf.to_crs(7803)
population_gdf.to_file("data/bulgaria_population.gpkg")